In [36]:
import os
import requests
from pytubefix import YouTube
import sys
import time
import re
from pprint import pprint

In [37]:
def youtube_download(url, path="."):
    yt = YouTube(url=url)
    print(yt.title)

    dl_yt_thumbnail(yt)

    dl_yt_audio_mp3(yt)

    return 0


def dl_yt_thumbnail(yt: YouTube):
    thumbnail_url = yt.thumbnail_url

    thumbnail_response = requests.get(thumbnail_url)

    if thumbnail_response.status_code == 200:
        thumbnail_file = os.path.join(".", "thumbnail.jpg")
        with open(thumbnail_file, "wb") as f:
            f.write(thumbnail_response.content)
        print("Thumbnail saved!")
    else:
        print(f"Error: {thumbnail_response.status_code}")


def dl_yt_audio_mp3(yt: YouTube):
    audio_stream = yt.streams.filter(only_audio=True).first()
    audio_dl = yt.streams.get_audio_only()
    audio_dl.download(filename="audio_test", mp3=True)

In [38]:
youtube_url = "https://www.youtube.com/watch?v=jTrBGdoehPI"

In [39]:
youtube_download(youtube_url)

Frequent flyer’s travel EDC (Amazon Must Haves)
Thumbnail saved!


0

In [50]:
from dotenv import load_dotenv

load_dotenv()

True

In [51]:
from openai import OpenAI

In [53]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [58]:
stream = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

This is a test.

In [59]:
from openai.types.audio import (
    Transcription,
    TranscriptionSegment,
    TranscriptionVerbose,
    TranscriptionWord,
    TranscriptionCreateResponse,
)

In [63]:
from pathlib import Path

In [66]:
pprint(client.models.list())

SyncPage[Model](data=[Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system'), Model(id='chatgpt-4o-latest', created=1723515131, object='model', owned_by='system'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='text-embedding-3-large', created=1705953180, object='model', owned_by='system'), Model(id='tts-1', created=1681940951, object='model', owned_by='openai-internal'), Model(id='tts-1-1106', created=1699053241, object='model', owned_by='system'), Model(id='gpt-4-0125-preview', created=1706037612, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-0125', created=1706048358, object='model', owned_by='system'), Model(id='gpt-4-turbo-preview', created=1706037777, object='model', owned_by='system'), Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'), Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'), Model(id='gpt-3.5-turbo-16k', created=168375

In [69]:
transcription_response = client.audio.transcriptions.create(
    file=Path("audio_test.mp3"),
    language="en",
    model="whisper-1",
)

In [70]:
transcription_response.text

"It's the first time ever I order Uber Eats this early. But you see I'm a bit tight on time because I have to pack. Apparently I'm going to Jackson Hole for a Polestar campaign and it's got nothing to do with this video but this being the 20th time I leave this year I figured I would show you my perfect minimal flying everyday carry bag. I've been on so many flights this year and to be honest I've developed this obsession with how to make my travels easier on me which is why I literally came up with a minimal flying everyday carry bag I can bring 99% of items from Amazon but this to me is useful whenever I decide to transport my camera and a backpack. Truthfully I really do wish I could bring you guys all the time to these type of events. Let's take a look at what I packed to get here and how my flight time went. So this here is codename 154 City Pack. The only item here that doesn't sell on Amazon. I just recently brought this with me to Berlin. I've gotta say it's incredibly well sor

In [71]:
summary_response = client.chat.completions.create(
    model="chatgpt-4o-latest",
    messages=[
        {"role": "user", "content": "Summarize the following text:"},
        {"role": "user", "content": transcription_response.text},
    ],
    stream=True,
)



In [74]:
for chunk in summary_response:
    print(chunk.choices[0].delta.content or "", end="")

The text describes the author's experience preparing for a trip, specifically focusing on their minimal everyday carry (EDC) kit designed for hassle-free flights. The author explains that they've been on many flights this year and have developed a preference for simple and efficient packing. They detail various items they carry, especially highlighting a minimal Italian leather bag they use — the 154 City Pack — which is not available on Amazon but has great capacity and thoughtful design. Alongside the bag, the author discusses electronics like their Pixel 9 Pro XL phone (praising its display, battery life, and camera), accessories like a Caseology phone case and MOFT wallet stand, and travel essentials such as the Bose QuietComfort Ultra headphones, X-Real AR glasses for immersive media consumption, and an Anker 3-in-1 power bank. The text emphasizes the practicality of these items in making travel more convenient, with a focus on lightweight, versatile gear often available on Amazon

In [40]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [41]:
torch.cuda.is_available()

True

In [42]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True,
)

In [43]:
result = pipe("audio_test.mp3")

In [44]:
pprint(result["text"])

(" It's the first time ever I order Uber Eats this early. But you see, I'm a "
 "bit tight on time because I have to pack. Apparently, I'm going to Jackson "
 "Hole for a Polestar campaign, and it's got nothing to do with this video. "
 'But this being the 20th time I leave this year, I figured I would show you '
 "my perfect minimal flying everyday carry. I've been on so many flights this "
 "year and to be honest I've developed this obsession with how to make my "
 'travels easier on me which is why I literally came up with a minimal flying '
 'everyday carry bag I can bring 99% of items from Amazon but this to me is '
 'useful whenever I decide to transport my camera and a backpack. Sometimes I '
 'really wonder how the heck we got here. Truthfully I really do wish I could '
 "bring you guys all the time to these type of events. Anyways, let's take a "
 'look at what I packed to get here and how my flight time went. So this here '
 "is codename 154 City Pack. The only item here that

In [45]:
import ollama

In [46]:
system_prompt = "Summarize the following transcript from a youtube video: '"
system_prompt += result["text"]

In [47]:
response = ollama.chat(
    model="llama3.1", messages=[{"role": "user", "content": system_prompt}]
)

In [48]:
pprint(response)

{'created_at': '2024-10-03T13:02:48.6616792Z',
 'done': True,
 'done_reason': 'stop',
 'eval_count': 331,
 'eval_duration': 119761465000,
 'load_duration': 3067088600,
 'message': {'content': "It seems like you've provided a review of various "
                        'travel-related items, including headphones (Bose), '
                        'glasses (Xreal Air 2s), and a power bank (Anker '
                        "3-in-1). Here's a summary of the key points:\n"
                        '\n'
                        '**Headphones: Bose**\n'
                        '\n'
                        '* The reviewer chose Bose because of their excellent '
                        'noise cancellation technology.\n'
                        '* They appreciate how the headphones fold nicely, are '
                        'comfortable to wear for extended periods, and have a '
                        'reliable connection with multiple devices at once.\n'
                        '* The reviewer lov

In [49]:
pprint(response["message"]["content"])

("It seems like you've provided a review of various travel-related items, "
 'including headphones (Bose), glasses (Xreal Air 2s), and a power bank (Anker '
 "3-in-1). Here's a summary of the key points:\n"
 '\n'
 '**Headphones: Bose**\n'
 '\n'
 '* The reviewer chose Bose because of their excellent noise cancellation '
 'technology.\n'
 '* They appreciate how the headphones fold nicely, are comfortable to wear '
 'for extended periods, and have a reliable connection with multiple devices '
 'at once.\n'
 '* The reviewer loves watching movies on these headphones, even though they '
 'prefer using a bigger screen like the Beam Pro.\n'
 '\n'
 '**Glasses: Xreal Air 2s**\n'
 '\n'
 "* These glasses use Sony's micro-OLED displays with 1080p per eye resolution "
 'and support up to 120Hz refresh rate.\n'
 '* They can project a 130-inch image with a 46-degree field of view, making '
 'them suitable for immersive experiences like movies or games.\n'
 '* The reviewer likes the built-in shade feat